Download historical data from EIA and NOAA/GHCN-d to the local filesystem

In [ ]:
import download_historical_data as dl
import os 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Importing some stuff from the FastAI book
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
import fastai.tabular.all as aiTab
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor, export_graphviz
import dtreeviz.trees as dtrees
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 10
pd.options.display.max_columns = 6

HISTORICAL_DATA_DIR = os.path.abspath("./historical_data")
ANALYSIS_DATA_DIR = os.path.abspath("./analysis_data/")
ELECTRIC_DATA_DIR = os.path.join(HISTORICAL_DATA_DIR, "electric_data")
WEATHER_DATA_DIR = os.path.join(HISTORICAL_DATA_DIR, "weather_station_data")

for dir in [HISTORICAL_DATA_DIR, ANALYSIS_DATA_DIR, ELECTRIC_DATA_DIR, WEATHER_DATA_DIR]:
    if not os.path.exists(dir):
        os.makedirs(dir)


WEATHER_STATION_IDS = [
    "USW00023066",  # Grand Junction Walker Field
    "USC00053553",  # Greeley UNC
    "USC00053005",  # Ft Collins
    "USC00050848",  # Boulder
    "USC00055984",  # Northglenn
    "USC00058995",  # Wheat Ridge
    "USW00023061"  # Alamosa
]

# Uncomment following lines to force re-download of source data
# Otherwise can also run the download script manually via: python download_historical_data.py
# Data files are saved locally so you only need to re-download to get new/different data

#dl.download_eia_historical_data(ELECTRIC_DATA_DIR, eia_respondent="PSCO")
#dl.download_ghcnd_historical_data(WEATHER_DATA_DIR, WEATHER_STATION_IDS)

In [ ]:
import glob

plt.style.use("default") #alternative "ggplot"

temp_df : pd.DataFrame = None

## Load up temperature data for each weather station, into their own columns
for df_file in glob.glob(WEATHER_DATA_DIR + "\*.json"):
    with open(df_file, "r", encoding="utf-8") as f:
        station_id = os.path.basename(df_file)[0:11]
        station_df = pd.read_json(f)
        station_df.index.rename("date", inplace=True)
        
        # TODO: This name-mangling seems like a halfassed way to either do a MultiIndex or maybe a tuple-index
        # Going to leave it for now as I'm not clear what will be easiest when trying to train an ML model
        col_renames = {col: f"{station_id}_{col}" for col in station_df.columns}
        station_df.rename(col_renames, axis="columns", inplace=True)
        
        if temp_df is not None:
            temp_df = pd.merge(left=temp_df, right=station_df, how="outer", left_index=True, right_index=True)
        else:
            temp_df = station_df

len(temp_df)

Load PSCO electric demand data from EIA

In [ ]:
psco_demand_data_file = os.path.join(ELECTRIC_DATA_DIR, "psco-daily-dataframe.json")
with open(psco_demand_data_file, "r", encoding="utf-8") as f:
    demand_df = pd.read_json(f)

len(demand_df)

Merge demand and temperature data

In [ ]:
joined_df = pd.merge(demand_df, temp_df, how="outer", left_index=True, right_index=True)
joined_df.dropna(inplace=True)
len(joined_df),joined_df.columns

Augment data with new dates and maybe some other stuff

In [ ]:
## Augment data
augmented_df = joined_df.copy()

# Extract date index into a column
augmented_df.reset_index(inplace=True)
augmented_df["date"] = augmented_df["index"]  
augmented_df.set_index("index", inplace=True)

## Adds date parts
augmented_df = aiTab.add_datepart(augmented_df, "date", drop=True)
## But a lot of the augmented parts are not that applicable in our case
augmented_df.drop(['Elapsed', 'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start',
                   'Is_year_end', 'Is_year_start'], axis=1, inplace=True)

## Add lagged values
# for lag in range(1, 15):
#     augmented_df[f"demand_lag_{lag}"] = augmented_df["daily_demand"].shift(lag)

augmented_df.columns

In [ ]:
# Create masks for data sets
train_mask = (augmented_df.Year < 2021)
validation_mask = ((augmented_df.Year >= 2021) & (augmented_df.Year < 2022))
test_mask = (augmented_df.Year >= 2022)

## Split out training, test and validation sets
train_df = augmented_df.where(train_mask).dropna()
validation_df = augmented_df.where(validation_mask).dropna()
test_df = augmented_df.where(test_mask).dropna()

# print(train_df.iloc[0:5][["daily_demand"]])
# print(validation_df.iloc[0:5][["daily_demand"]])
# print(test_df.iloc[0:5])

In [ ]:
## Create indexes from the sets
train_idx = np.where(train_mask)[0]
valid_idx = np.where(validation_mask)[0]
test_idx = np.where(test_mask)[0]
print(f"trainSize={len(train_idx)}, validationSize={len(valid_idx)}, testSize={len(test_idx)}")
# print(train_idx[0:5])
# print(valid_idx[0:5])
# print(test_idx[0:5])

splits = (list(train_idx), list(valid_idx))

In [ ]:
## Split out categorical vs continuous data
cont, cat = aiTab.cont_cat_split(augmented_df, 1, dep_var="daily_demand")
print(cont)
print(cat)

In [ ]:
## Create TabularPandas
procs = [aiTab.Categorify, aiTab.FillMissing]
dep_var = "daily_demand"
to = aiTab.TabularPandas(augmented_df, procs, cat, cont, y_names=dep_var, splits=splits)
len(to.train),len(to.valid)

In [ ]:
to.show(3)

In [ ]:
aiTab.save_pickle(os.path.join(ANALYSIS_DATA_DIR, 'tabular.pkl'), to)

In [ ]:
## Reload from pickle
to = aiTab.load_pickle(os.path.join(ANALYSIS_DATA_DIR, 'tabular.pkl'))

Pick out the validation input and output data (and duplicate that effect with the test Dataframes)

In [ ]:
xs,y = to.train.xs,to.train.y
valid_xs,valid_y = to.valid.xs,to.valid.y
test_xs = test_df.drop("daily_demand", axis=1, inplace=False)
test_y = test_df["daily_demand"]

In [ ]:
## Create a decision tree
tree = DecisionTreeRegressor(max_leaf_nodes=4)
tree.fit(xs, y)

In [ ]:
from sklearn.tree import export_graphviz
import graphviz
import re

def draw_tree(t, df, size=10, ratio=0.6, precision=0, **kwargs):
    s = export_graphviz(t, out_file=None, feature_names=df.columns, filled=True, rounded=True,
                        special_characters=True, rotate=False, precision=precision, **kwargs)
    return graphviz.Source(re.sub('Tree {', f'Tree {{ size={size}; ratio={ratio}', s))


# Draw the tree
draw_tree(tree, xs, size=10, leaves_parallel=True, precision=2)

In [ ]:
## Let's see that in DTreeViz
samp_idx = np.random.permutation(len(y))[:500]
dtrees.dtreeviz(tree, xs.iloc[samp_idx], y.iloc[samp_idx], xs.columns, dep_var,
         fontname='DejaVu Sans', scale=1.6, label_fontsize=10,
         orientation='LR')

In [ ]:
## MOAR LEAVES
tree = DecisionTreeRegressor(min_samples_leaf=15)
tree.fit(xs, y)

In [ ]:
## Functions to check root-mean-squared error for the model
import math
def r_mse(pred, y): return round(math.sqrt(((pred - y)**2).mean()), 6)
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)

In [ ]:
## Check error in the training set
m_rmse(tree, xs, y)

In [ ]:
## Check RMS error against validation set
m_rmse(tree, valid_xs, valid_y)

In [ ]:
## How many leaves do we have? Oh, it's one per measurement, so massively overfitted
tree.get_n_leaves(), len(xs)

In [ ]:
# Draw the full tree (use carefully, it's pretty big)
# samp_idx = np.random.permutation(len(y))[:500]
# dtrees.dtreeviz(tree, xs.iloc[samp_idx], y.iloc[samp_idx], xs.columns, dep_var,
#                 fontname='DejaVu Sans', scale=1.6, label_fontsize=10,
#                 orientation='LR')

In [ ]:
## Function to grow a random forest with some default parameters chosen

## n_estimators -> number of trees in the forest
num_estimators = 200

def grow_random_forest(xs, y, n_estimators=num_estimators, max_samples=0.8,
       max_features=0.5, min_samples_leaf=4, **kwargs):
   m = RandomForestRegressor(n_jobs=-1, n_estimators=n_estimators,
                                 max_samples=max_samples, max_features=max_features,
                                 min_samples_leaf=min_samples_leaf, oob_score=True)
   return m.fit(xs, y)

In [ ]:
forest = grow_random_forest(xs, y)

In [ ]:
m_rmse(forest, xs, y), m_rmse(forest, valid_xs, valid_y)

In [ ]:
## Get predictions from each individual tree in the forest
preds = np.stack([t.predict(valid_xs) for t in forest.estimators_])

## Plot the mean error for a given number of estimators used
plt.plot([r_mse(preds[:i + 1].mean(0), valid_y) for i in range(num_estimators)])


In [ ]:
# Out-of-bag errors for the forest
r_mse(forest.oob_prediction_, y)

In [ ]:
# Get the standard deviation of the estimates from each tree, for each row in the validation set
preds_std = preds.std(0)

#std dev for the first 5 rows
preds_std[:5]

In [ ]:
def rf_feature_importance(m, df):
    return pd.DataFrame({'cols': df.columns, 'imp': m.feature_importances_}
                        ).sort_values('imp', ascending=False)

In [ ]:
fi = rf_feature_importance(forest, xs)
fi[:10]

In [ ]:
def plot_feature_importance(fi):
    return fi.plot('cols', 'imp', 'barh', figsize=(12, 7), legend=False)


plot_feature_importance(fi[:30])

In [ ]:
## Trim out some low-importance columns
to_keep = fi[fi.imp > 0.005].cols
len(fi.cols),len(to_keep)

In [ ]:
# Create new & improved datasets (without the low-importance columns)
xs_imp = xs[to_keep]
valid_xs_imp = valid_xs[to_keep]
test_xs_imp = test_xs[to_keep]

In [ ]:
# retrain
forest = grow_random_forest(xs_imp, y)

In [ ]:
# retest
m_rmse(forest, xs_imp, y), m_rmse(forest, valid_xs_imp, valid_y)

In [ ]:
## Replot feature importance
plot_feature_importance(rf_feature_importance(forest, xs_imp))

In [ ]:
## Function to draw rank importance diagrams, stolen from the FastAI book code
import scipy
import scipy.cluster.hierarchy as hc

def cluster_columns(df, figsize=(10, 6), font_size=12):
    corr = np.round(scipy.stats.spearmanr(df).correlation, 4)
    corr_condensed = hc.distance.squareform(1 - corr)
    z = hc.linkage(corr_condensed, method='average')
    fig = plt.figure(figsize=figsize)
    hc.dendrogram(z, labels=df.columns, orientation='left', leaf_font_size=font_size)
    plt.show()

cluster_columns(xs_imp)

In [ ]:
## Get out-of-bag scores for a fairly simple model
## Although this one is so fast to train that we're not going to make it any simpler
def get_oob(df, y):
    ## Original code from FastAI book
    # m = RandomForestRegressor(n_estimators=40, min_samples_leaf=15,
    #                           max_samples=50000, max_features=0.5, n_jobs=-1, oob_score=True)
    m = grow_random_forest(df, y)
    return m.oob_score_

In [ ]:
## Baseline
get_oob(xs_imp, y)

In [ ]:
## Try removing potentially redundant values one at a time
{c:get_oob(xs_imp.drop(c, axis=1), y) for c in (
    'Week', 'Dayofyear', 
    'USC00058995_tmax', 'USC00055984_tmax',
    'USC00053005_tmax', 'USC00050848_tmax',
    'USC00053005_tmin', 'USC00053553_tmin',
    
    # This is actually our _most_ important column, but I kinda hate it for predictions.
    # So I want to see what happens if we remove it
    'Year'
    )}

In [ ]:
## Drop out one from each pair of redundant columns and retest
to_drop = ['Week', #'Dayofyear',
           'USC00058995_tmax', #'USC00055984_tmax',
           'USC00053005_tmax', #'USC00050848_tmax',
           'USC00053005_tmin', #'USC00053553_tmin'
           ]
get_oob(xs_imp.drop(to_drop, axis=1), y)

In [ ]:
## Create DFs without the redundant columns
xs_final = xs_imp.drop(to_drop, axis=1)
valid_xs_final = valid_xs_imp.drop(to_drop, axis=1)
test_xs_final = test_xs_imp.drop(to_drop, axis=1)

aiTab.save_pickle(os.path.join(ANALYSIS_DATA_DIR, 'xs_final.pkl'), xs_final)
aiTab.save_pickle(os.path.join(ANALYSIS_DATA_DIR, 'valid_final.pkl'), valid_xs_final)
aiTab.save_pickle(os.path.join(ANALYSIS_DATA_DIR, 'test_final.pkl'), test_xs_final)


In [ ]:
## Reload data from the pickles. Unnecessary since we just created it, but we could skip the above steps if desired.
# xs_final = aiTab.load_pickle(os.path.join(ANALYSIS_DATA_DIR, 'xs_final.pkl'))
# valid_xs_final = aiTab.load_pickle(os.path.join(ANALYSIS_DATA_DIR, 'valid_final.pkl'))
# test_xs_final = aiTab.load_pickle(os.path.join(ANALYSIS_DATA_DIR, 'test_final.pkl'))

In [ ]:
## Grow a new forst with our final dataset
forest = grow_random_forest(xs_final, y)
m_rmse(forest, xs_final, y), m_rmse(forest, valid_xs_final, valid_y)

In [ ]:
## Recheck errors
m_rmse(forest, xs_final, y), m_rmse(forest, valid_xs_final, valid_y)

In [ ]:
## Check the partial dependence on each variable
from sklearn.inspection import plot_partial_dependence

fig, ax = plt.subplots(figsize=(12, 4))
# plot_partial_dependence(forest, valid_xs_final, ['Year', 'Dayofweek'],
#                         grid_resolution=20, ax=ax)
plot_partial_dependence(forest, xs_final, ['Year', 'Dayofyear', 'Dayofweek'],
                        grid_resolution=20, ax=ax)


In [ ]:
from waterfall_chart import plot as waterfall
from treeinterpreter import treeinterpreter
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [ ]:
row = valid_xs_final
prediction,bias,contributions = treeinterpreter.predict(forest, row.values)
prediction[0], bias[0], contributions[0].sum()

Graph the contributions of each parameter to the first row of data 

In [ ]:
waterfall(valid_xs_final.columns, contributions[0], threshold=0.08, 
          rotation_value=90,formatting='{:,.3f}');

Graph the mean contributions of each parameter across the entire dataset

In [ ]:
# Explode contributions into a dataframe
contributions_df = pd.DataFrame(contributions, columns=valid_xs_final.columns)
means = contributions_df.mean()
waterfall(index=means.index, data=means, threshold=0.00, 
          rotation_value=90,formatting='{:,.3f}')

In [ ]:
## Check errors on our test set
#prediction, bias, contributions = treeinterpreter.predict(forest, row.values)
m_rmse(forest, test_xs_final, test_y)

In [ ]:
## Graph test set predictions vs actuals
y_pred = forest.predict(test_xs_final)
pred_df = pd.DataFrame(data=y_pred, index=list(test_xs_final.index))
pred_df = pred_df.join(test_y, how="inner")
pred_df.rename(columns={0: "predicted_demand", "daily_demand":"actual_demand"}, inplace=True)
pred_df.plot()